In [1]:
import pandas as pd
import math
import scipy
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import cptac
import cptac.utils as ut
import copy
import numpy as np

In [2]:
brca = cptac.Brca()
ccrcc = cptac.Ccrcc()
colon = cptac.Colon()
en = cptac.Endometrial()
gbm = cptac.Gbm()
luad = cptac.Luad()
ovarian = cptac.Ovarian()
hnscc = cptac.Hnscc()
lscc = cptac.Lscc()

cptac warning: The LSCC data is currently strictly reserved for CPTAC investigators. Otherwise, you are not authorized to access these data. Additionally, even after these data become publicly available, they will be subject to a publication embargo (see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details). (<ipython-input-2-ec2884862cde>, line 9)


# Get dataframes to correct

In [3]:
perm_corrected = pd.read_csv("full_10k_permutation_corrected.csv")
corr_diff = pd.read_csv("corr_diff.csv")
corr_df = pd.read_csv("correlations_dataframe.csv")#this one wait
pval_df = pd.read_csv("p_val.csv")
all_genes = (list(pd.read_csv("gene_list.csv").columns))

# Get tumor and normal data to check minimum sample number

In [4]:
ccrcc_prot_trans_tumor = (ut.reduce_multiindex(df=ccrcc.join_omics_to_omics(df1_name="proteomics", df2_name="transcriptomics", tissue_type="tumor"), levels_to_drop="Database_ID"))
ccrcc_prot_trans_normal = (ut.reduce_multiindex(df=ccrcc.join_omics_to_omics(df1_name="proteomics", df2_name="transcriptomics", tissue_type="normal"), levels_to_drop="Database_ID"))

ccrcc_prot_trans_tumor = ccrcc_prot_trans_tumor.loc[:,~ccrcc_prot_trans_tumor.columns.duplicated()]
ccrcc_prot_trans_normal = ccrcc_prot_trans_normal.loc[:,~ccrcc_prot_trans_normal.columns.duplicated()]



cptac warning: Due to dropping the specified levels, dataframe now has 907 duplicated column headers. (<ipython-input-4-e211584053b8>, line 1)
cptac warning: transcriptomics data was not found for the following samples, so transcriptomics data columns were filled with NaN for these samples: C3L-00183.N, C3L-00791.N, C3N-00246.N, C3N-00312.N, C3N-00317.N, C3N-00320.N, C3N-00437.N, C3N-00492.N, C3N-00953.N (<ipython-input-4-e211584053b8>, line 2)
cptac warning: Due to dropping the specified levels, dataframe now has 907 duplicated column headers. (<ipython-input-4-e211584053b8>, line 2)


In [5]:
en_prot_trans_tumor = en.join_omics_to_omics(df1_name="proteomics", df2_name="transcriptomics", tissue_type="tumor")
en_prot_trans_normal = en.join_omics_to_omics(df1_name="proteomics", df2_name="transcriptomics", tissue_type="normal")


cptac warning: transcriptomics data was not found for the following samples, so transcriptomics data columns were filled with NaN for these samples: C3L-00563.N, C3L-00605.N, C3L-00769.N, C3L-00770.N, C3L-00771.N, C3L-00930.N, C3L-00947.N, C3L-00963.N, C3L-01246.N, C3L-01249.N, C3L-01252.N, C3L-01256.N, C3L-01257.N, C3L-01744.N, C3N-00200.N, C3N-00729.N, C3N-01211.N, NX1.N, NX10.N, NX11.N, NX12.N, NX13.N, NX14.N, NX15.N, NX16.N, NX17.N, NX18.N, NX2.N, NX3.N, NX4.N, NX5.N, NX6.N, NX7.N, NX8.N, NX9.N (<ipython-input-5-667cf0d856a9>, line 2)


In [6]:
luad_prot_trans_tumor = ut.reduce_multiindex(df=luad.join_omics_to_omics(df1_name="proteomics", df2_name="transcriptomics", tissue_type="tumor"), levels_to_drop="Database_ID")
luad_prot_trans_normal = ut.reduce_multiindex(df=luad.join_omics_to_omics(df1_name="proteomics", df2_name="transcriptomics", tissue_type="normal"), levels_to_drop="Database_ID")

luad_prot_trans_tumor = luad_prot_trans_tumor.loc[:,~luad_prot_trans_tumor.columns.duplicated()]
luad_prot_trans_normal = luad_prot_trans_normal.loc[:,~luad_prot_trans_normal.columns.duplicated()]



cptac warning: Due to dropping the specified levels, dataframe now has 745 duplicated column headers. (<ipython-input-6-a251c483c914>, line 1)
cptac warning: Due to dropping the specified levels, dataframe now has 745 duplicated column headers. (<ipython-input-6-a251c483c914>, line 2)


In [7]:
hnscc_prot_luad_tumor = hnscc.join_omics_to_omics(df1_name="proteomics", df2_name="transcriptomics", tissue_type="tumor")
hnscc_prot_luad_normal = hnscc.join_omics_to_omics(df1_name="proteomics", df2_name="transcriptomics", tissue_type="normal")



cptac warning: transcriptomics data was not found for the following samples, so transcriptomics data columns were filled with NaN for these samples: C3L-00994.C, C3L-02617.C, C3L-04350.C, C3L-04354.N, C3L-04844.N, C3L-05257.C, C3N-00295.N, C3N-00829.N, C3N-00857.N, C3N-01620.N, C3N-01757.C, C3N-01758.N, C3N-01858.N, C3N-01859.N, C3N-02693.N, C3N-03008.N, C3N-03011.N, C3N-03012.N, C3N-03013.N, C3N-03015.N, C3N-03042.C, C3N-03781.N, C3N-04275.N, C3N-04276.N, C3N-04277.N, C3N-04278.N, C3N-04279.N (<ipython-input-7-31a112e48ffa>, line 2)
cptac warning: proteomics data was not found for the following samples, so proteomics data columns were filled with NaN for these samples: C3L-00977.N, C3L-04025.N, C3N-01948.N, C3N-02333.N, C3N-02727.N, C3N-02925.N, C3N-03226.N, C3N-03487.N, C3N-03889.N, C3N-04152.N, C3N-04611.N (<ipython-input-7-31a112e48ffa>, line 2)


In [8]:
lscc_prot_trans_tumor = ut.reduce_multiindex(df=lscc.join_omics_to_omics(df1_name="proteomics", df2_name="transcriptomics", tissue_type="tumor"), levels_to_drop="Database_ID")
lscc_prot_trans_normal = ut.reduce_multiindex(df=lscc.join_omics_to_omics(df1_name="proteomics", df2_name="transcriptomics", tissue_type="normal"), levels_to_drop="Database_ID")


lscc_prot_trans_tumor = lscc_prot_trans_tumor.loc[:,~lscc_prot_trans_tumor.columns.duplicated()]
lscc_prot_trans_normal = lscc_prot_trans_normal.loc[:,~lscc_prot_trans_normal.columns.duplicated()]



cptac warning: Due to dropping the specified levels, dataframe now has 858 duplicated column headers. (<ipython-input-8-fefd7eb15fae>, line 1)
cptac warning: transcriptomics data was not found for the following samples, so transcriptomics data columns were filled with NaN for these samples: C3L-02646.N, C3N-03072.N, C3N-03662.N, C3N-03886.N, C3N-04155.N (<ipython-input-8-fefd7eb15fae>, line 2)
cptac warning: Due to dropping the specified levels, dataframe now has 858 duplicated column headers. (<ipython-input-8-fefd7eb15fae>, line 2)


In [9]:
tissue_list = [[ccrcc_prot_trans_tumor,ccrcc_prot_trans_normal],[en_prot_trans_tumor, en_prot_trans_normal],[luad_prot_trans_tumor, luad_prot_trans_normal],[hnscc_prot_luad_tumor, hnscc_prot_luad_normal],[lscc_prot_trans_tumor, lscc_prot_trans_normal]]
order = ["ccrcc", "en", "luad", "hnscc", "lscc"]


# Function to fix dataframe

In [10]:
def fix_df(df, genes, tissues, order):

    correct_dp = copy.deepcopy(df.set_index("Cancer"))             
    data = {}
    data["Cancer"] = order
    genes_in_df = correct_dp.columns
    for i in genes_in_df:
        data[i] = []
    
    for i in genes_in_df:
        for index, value in enumerate(tissues):
            if (i+"_proteomics") in value[0].columns and (i+"_transcriptomics") in value[0].columns and (i+"_proteomics") in value[1].columns and (i+"_transcriptomics") in value[1].columns:
                if (len(value[0][i+"_proteomics"].dropna())) > 10 and (len(value[0][i+"_transcriptomics"].dropna())) and (len(value[1][i+"_proteomics"].dropna())) and (len(value[1][i+"_transcriptomics"].dropna())):
                    data[i].append(correct_dp.iloc[index][i])
                else:
                    data[i].append(np.nan)
            else:
                    data[i].append(np.nan)
                    
    return(pd.DataFrame(data))



# Dataframe output

In [11]:
corr_diff

,Cancer,A1BG,A1CF,A2M,A2ML1,A4GALT,AAAS,AACS,AADAC,AADAT,...,ZSWIM9,ZW10,ZWILCH,ZWINT,ZXDA,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
0,ccrcc,-0.388745,0.146993,-0.277035,NaN,NaN,-0.044291,-0.128218,NaN,-0.174925,...,NaN,0.269728,0.329727,-0.337301,NaN,0.337470,0.109414,-0.223934,-0.138352,-0.220727
1,en,0.061654,NaN,-0.025535,0.402210,0.399061,0.663703,0.593777,NaN,0.302879,...,0.177842,0.088061,0.759429,1.006282,NaN,-0.267078,0.400272,0.147305,0.367791,0.342423
2,luad,-0.039149,NaN,0.097233,NaN,NaN,0.206094,0.683092,-0.119746,0.630155,...,NaN,0.557832,0.904669,0.645076,NaN,0.087794,0.707991,0.472139,0.418589,0.661323
3,hnscc,-0.002651,0.407295,-0.194186,0.117838,0.555675,0.462864,0.024965,-0.333934,NaN,...,NaN,0.395189,0.119698,1.079145,NaN,0.591841,-0.453269,-0.010672,0.255505,-0.060708
4,lscc,0.196946,NaN,-0.062630,0.649123,0.246831,0.304418,0.150665,-0.068258,NaN,...,0.613891,0.491043,0.646430,0.838981,NaN,0.332350,0.231167,0.315679,0.268239,0.446553


In [12]:
perm_corrected = fix_df(perm_corrected, all_genes, tissue_list, order).set_index("Cancer")
corr_diff = fix_df(corr_diff, all_genes, tissue_list, order).set_index("Cancer")
pval_df = fix_df(pval_df, all_genes, tissue_list, order).set_index("Cancer")


In [13]:
new_index_corr_diff = list(corr_diff.index)
new_index_corr_diff[1] = "endometrial"
corr_diff.index = new_index_corr_diff


In [14]:
perm_corrected.to_csv("full_10k_permutation_corrected.csv")
corr_diff.to_csv("corr_diff.csv")
pval_df.to_csv("p_val.csv")
